In [25]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import codecs
import time
import re

In [26]:
from IPython.display import display, HTML

In [27]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

In [28]:
# headers = {
#     'User-Agent': 'Your Name, example.com',
#     'From': 'email@example.com'
# }

In [79]:
fr = open("workdata/linesdata/sansk.txt",'r')
_lines = fr.readlines()
fr.close()
count = 1
for _line in _lines :
    fw = codecs.open("workdata/linetabledata/"+ str(count)+".txt",encoding = 'utf-8',mode = 'w')
    time.sleep(5)
    inputline = re.match(r'\d+-(.*)',_line).group(1)
    s_d = inputline
    s_c = s_d.replace(" ","+")
    urlname = ("http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=" +
               s_c + "&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=")

    print(urlname)
    page = requests.get(urlname)
    soup = bs(page.text, 'html.parser')
#     soup.prettify()
    table = soup.table
    tablebody = table.find('table',{'class' : 'center'})
    t = pd.DataFrame(columns = ['id','level','color_class','position','chunk_no','word','lemma','pre_verb','morph','colspan','wordlenth','aux_inf'])

    i = 0
    id_= 0
    for child in tablebody.children:
        if(child.name == 'tr') :
            if i< 1 : 
                linechar = []
                c=0
                for char in child.children :
                    linechar.append(char.string)
                    c += 1
                i+=1
                line = "".join(linechar)
                linechunks = line.split("\xa0")
                continue
            position_ = 0
            j =0
            for wordtable in child.children:
    #             if(j < 20 ) :
    #                 j += 1
    #                 continue
                c= 0
                for ch in linechar[0:position_] :
                    if(re.match('\xa0',ch)) : 
                        c+=1
                if(wordtable.contents):
                    color_ = wordtable.table.get('class')[0]
                    colspan_= wordtable.get('colspan')
                    word_ = wordtable.table.tr.td.string
                    onclickdatas_ = wordtable.table.tr.td.get('onclick')
                    for onclickdata_ in onclickdatas_.split("<br>") :
                        morphslist_ = re.findall(r'{ (.*?) }',onclickdata_)  #.split(' | ')
                        ldata = str(re.search(r'{.*?}\[(.*)\]',onclickdata_).group(1))
                        ldata = str(re.sub(r'</?a.*?>|</?i>',"",ldata))

                        lemmadata = ldata.split(" ")
                        if len(lemmadata) >1 :
                            auxi_ = " ".join(lemmadata[1:])
                        else :
                            auxi_ = ""
                        lemmas_ = "".join(lemmadata[0])
                        lemmalists_ = lemmas_.split("-")
                        if(len(lemmalists_) > 1) :
                            preverb_ = ",".join(lemmalists_[0:(len(lemmalists_)-1)])
                            lemmalist_ = "".join(lemmalists_[-1:]).split("_")
                        else :
                            preverb_ = ""
                            lemmalist_ = "".join(lemmalists_[0]).split("_")
                        if(len(lemmalist_) > 1):
                            auxi_ =auxi_ + " sence of lemma = " + "".join(lemmalist_[1:(len(lemmalist_))])
                            lemma_ = "".join(lemmalist_[0])
                        else : 
                            lemma_ = "".join(lemmalist_[0])             
                        morphs_ = str(morphslist_[0])               
                        for morph_ in morphs_.split(" | ") : 
                            t.loc[id_] = [id_,i,color_,position_,c+1,word_,lemma_,preverb_,morph_,colspan_,len(word_),auxi_]
                            id_ += 1    


                    position_ += int(colspan_)
        #                 break
                else : 
                    position_ += 1

    #         break

            i = i+1
    fw.writelines(inputline + "\n")
    fw.writelines(line + "\n")
    fw.writelines(",".join(linechar) + "\n")
    fw.close()
    n ="workdata/linetabledata/"+ str(count)+".txt"
    t.to_csv(n ,encoding = 'utf-8' , sep=',',index = False,mode = 'a' )
    display(t)
    count += 1 
#     if(count > 2) :
#         break
    

# fr = open("data/sansk.txt",'r')
# print(fr.readlines())
# fr.close()

http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=xaSaraWasya+puwraH+nAmnA+rAmaH&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=


,id,level,color_class,position,chunk_no,word,lemma,pre_verb,morph,colspan,wordlenth,aux_inf
0,0,1,yellow_back,0,1,daśa,daśan,,iic.,4,4,
1,1,1,deep_sky_back,4,1,rathasya,ratha,,g. sg. m.,7,8,
2,2,1,deep_sky_back,12,2,putraḥ,putra,,nom. sg. m.,6,6,
3,3,1,mauve_back,19,3,nāmnā,nāman,,ind.,5,5,
4,4,1,deep_sky_back,25,3,rāmaḥ,rāma,,nom. sg. m.,5,5,
5,5,2,deep_sky_back,0,1,daśa,daśan,,acc. pl. *,4,4,
6,6,2,deep_sky_back,0,1,daśa,daśan,,nom. pl. *,4,4,
7,7,2,cyan_back,4,1,rathasya,ratha,,g. sg. n.,7,8,
8,8,2,deep_sky_back,19,3,nāmnā,nāman,,i. sg. n.,5,5,
9,9,2,carmin_back,25,3,rāmaḥ,rā,,pr. [2] ac. pl. 1,5,5,sence of lemma = 1


http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=kRewragAminA+varwmanA+gacCanwaM+yAwrikaM+apaSyaw&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=


,id,level,color_class,position,chunk_no,word,lemma,pre_verb,morph,colspan,wordlenth,aux_inf
0,0,1,yellow_back,0,1,kṣetra,kṣetra,,iic.,6,6,
1,1,1,deep_sky_back,6,1,gāminā,gāmin,,i. sg. m.,6,6,
2,2,1,deep_sky_back,6,1,gāminā,gāmin,,i. sg. n.,6,6,
3,3,1,deep_sky_back,13,1,vartmanā,vartman,,i. sg. n.,8,8,
4,4,1,deep_sky_back,22,1,gacchantam,gacchat,,acc. sg. m.,9,10,{ ppr. [1] ac. }[gam]
5,5,1,deep_sky_back,32,2,yātrikam,yātrika,,acc. sg. m.,8,8,
6,6,1,deep_sky_back,32,2,yātrikam,yātrika,,acc. sg. n.,8,8,
7,7,1,deep_sky_back,32,2,yātrikam,yātrika,,nom. sg. n.,8,8,
8,8,1,carmin_back,41,3,apaśyat,paś,,impft. [4] ac. sg. 3,7,7,
9,9,2,deep_sky_back,6,1,gām,go,,acc. sg. m.,3,3,


http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=janasya+kalyANAya+yawamAnena+rAmeNa+AwmA+kleSaM+upAnIyawa&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=


...

,id,level,color_class,position,chunk_no,word,lemma,pre_verb,morph,colspan,wordlenth,aux_inf
0,0,1,deep_sky_back,0,1,dvārakām,dvāraka,,acc. sg. f.,8,8,
1,1,1,mauve_back,9,2,adhi,adhi,,prep.,3,4,
2,2,1,deep_sky_back,12,2,ūṣuṣaḥ,ūṣivas,,g. sg. n.,6,6,{ ppf. ac. }[uṣ]
3,3,1,deep_sky_back,12,2,ūṣuṣaḥ,ūṣivas,,abl. sg. n.,6,6,{ ppf. ac. }[uṣ]
4,4,1,deep_sky_back,12,2,ūṣuṣaḥ,ūṣivas,,acc. pl. m.,6,6,{ ppf. ac. }[uṣ]
5,5,1,deep_sky_back,12,2,ūṣuṣaḥ,ūṣivas,,g. sg. m.,6,6,{ ppf. ac. }[uṣ]
6,6,1,deep_sky_back,12,2,ūṣuṣaḥ,ūṣivas,,abl. sg. m.,6,6,{ ppf. ac. }[uṣ]
7,7,1,deep_sky_back,19,3,janasya,jana,,g. sg. m.,7,7,
8,8,1,deep_sky_back,19,3,janasya,jana,,g. sg. n.,7,7,
9,9,1,deep_sky_back,27,4,sampadaḥ,sampad,,acc. pl. f.,8,8,sence of lemma = 2


http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=guNAH+pUjA-sWAnaM+guNiRu+na+ca+lifgaM+na+ca+vayaH&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=


,id,level,color_class,position,chunk_no,word,lemma,pre_verb,morph,colspan,wordlenth,aux_inf
0,0,1,deep_sky_back,0,1,guṇāḥ,guṇa,,nom. pl. m.,5,5,
1,1,1,yellow_back,6,2,pūjā,pūjā,,iic.,4,4,
2,2,1,deep_sky_back,10,2,sthānam,sthāna,,acc. sg. n.,6,7,
3,3,1,deep_sky_back,10,2,sthānam,sthāna,,nom. sg. n.,6,7,
4,4,1,deep_sky_back,18,3,guṇiṣu,guṇin,,loc. pl. m.,6,6,
5,5,1,deep_sky_back,18,3,guṇiṣu,guṇin,,loc. pl. n.,6,6,
6,6,1,mauve_back,25,4,na,na,,part.,2,2,
7,7,1,mauve_back,28,5,ca,ca,,conj.,2,2,
8,8,1,deep_sky_back,31,6,liṅgam,liṅga,,acc. sg. n.,6,6,
9,9,1,deep_sky_back,31,6,liṅgam,liṅga,,nom. sg. n.,6,6,


http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=kEkeyyAH+xAruNaM+vacaH+SruwvA+mahArAjaH+xaSaraWaH+sahasA+BUmO+apawaw+niSceRtaH+ca+aBavaw&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=


,id,level,color_class,position,chunk_no,word,lemma,pre_verb,morph,colspan,wordlenth,aux_inf
0,0,1,deep_sky_back,0,1,kaikeyyāḥ,kaikeya,,g. sg. f.,8,9,
1,1,1,deep_sky_back,0,1,kaikeyyāḥ,kaikeya,,abl. sg. f.,8,9,
2,2,1,deep_sky_back,9,2,dāruṇam,dāruṇa,,acc. sg. m.,7,7,
3,3,1,deep_sky_back,9,2,dāruṇam,dāruṇa,,acc. sg. n.,7,7,
4,4,1,deep_sky_back,9,2,dāruṇam,dāruṇa,,nom. sg. n.,7,7,
5,5,1,deep_sky_back,17,3,vacaḥ,vacas,,acc. sg. n.,5,5,
6,6,1,deep_sky_back,17,3,vacaḥ,vacas,,nom. sg. n.,5,5,
7,7,1,deep_sky_back,17,3,vacaḥ,vaca,,nom. sg. m.,5,5,
8,8,1,mauve_back,23,4,śrutvā,śru,,abs.,6,6,
9,9,1,mauve_back,30,4,mahaḥ,mahas,,ind.,5,5,


In [78]:
s = pd.read_csv(n,encoding = 'utf-8' ,sep = ',',skiprows = 3)
s

,id,level,color_class,position,chunk_no,word,lemma,pre_verb,morph,colspan,wordlenth,aux_inf
0,0,1,yellow_back,0,1,kṣetra,kṣetra,NaN,iic.,6,6,NaN
1,1,1,deep_sky_back,6,1,gāminā,gāmin,NaN,i. sg. m.,6,6,NaN
2,2,1,deep_sky_back,6,1,gāminā,gāmin,NaN,i. sg. n.,6,6,NaN
3,3,1,deep_sky_back,13,1,vartmanā,vartman,NaN,i. sg. n.,8,8,NaN
4,4,1,deep_sky_back,22,1,gacchantam,gacchat,NaN,acc. sg. m.,9,10,{ ppr. [1] ac. }[gam]
5,5,1,deep_sky_back,32,2,yātrikam,yātrika,NaN,acc. sg. m.,8,8,NaN
6,6,1,deep_sky_back,32,2,yātrikam,yātrika,NaN,acc. sg. n.,8,8,NaN
7,7,1,deep_sky_back,32,2,yātrikam,yātrika,NaN,nom. sg. n.,8,8,NaN
8,8,1,carmin_back,41,3,apaśyat,paś,NaN,impft. [4] ac. sg. 3,7,7,NaN
9,9,2,deep_sky_back,6,1,gām,go,NaN,acc. sg. m.,3,3,NaN


In [30]:
#     s_d = inputline
#     s_c = s_d.replace(" ","+")
#     urlname = ("http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=t&us=f&cp=t&text=" +
#                s_c + "&t=WX&topic=&mode=g&corpmode=&corpdir=&sentno=")

#     print(urlname)
#     time.sleep(2)
#     page = requests.get(urlname)

In [31]:
#     soup = bs(page.text, 'html.parser')
#     soup.prettify()

In [32]:
# table = soup.table
# print(table)

In [33]:
# tablebody = table.find('table',{'class' : 'center'})
# print(tablebody)

In [34]:
# t = pd.DataFrame(columns = ['id','level','color_class','position','chunk_no','word','lemma','pre_verb','morph','colspan','wordlenth','aux_inf'])
# t


In [35]:
# for child in tablebody.children:
#     if(child.name == 'tr') :
#         print(child.name)

In [36]:
# 'id','level','color_class','position','chunk_no','word','lemma','morph','colspan','wordlenth','pre_verb','aux_inf'

In [37]:
# i = 0
# id_= 0
# for child in tablebody.children:
#     if(child.name == 'tr') :
#         if i< 1 : 
#             linechar = []
#             c=0
#             for char in child.children :
#                 linechar.append(char.string)
#                 c += 1
#             i+=1
#             line = "".join(linechar)
#             linechunks = line.split("\xa0")
#             continue
#         position_ = 0
#         j =0
#         for wordtable in child.children:
# #             if(j < 20 ) :
# #                 j += 1
# #                 continue
#             c= 0
#             for ch in linechar[0:position_] :
#                 if(re.match('\xa0',ch)) : 
#                     c+=1
#             if(wordtable.contents):
#                 color_ = wordtable.table.get('class')[0]
#                 colspan_= wordtable.get('colspan')
#                 word_ = wordtable.table.tr.td.string
#                 onclickdatas_ = wordtable.table.tr.td.get('onclick')
#                 for onclickdata_ in onclickdatas_.split("<br>") :
#                     morphslist_ = re.findall(r'{ (.*?) }',onclickdata_)  #.split(' | ')
#                     ldata = str(re.search(r'{.*?}\[(.*)\]',onclickdata_).group(1))
#                     ldata = str(re.sub(r'</?a.*?>|</?i>',"",ldata))

#                     lemmadata = ldata.split(" ")
#                     if len(lemmadata) >1 :
#                         auxi_ = " ".join(lemmadata[1:])
#                     else :
#                         auxi_ = ""
#                     lemmas_ = "".join(lemmadata[0])
#                     lemmalists_ = lemmas_.split("-")
#                     if(len(lemmalists_) > 1) :
#                         preverb_ = ",".join(lemmalists_[0:(len(lemmalists_)-1)])
#                         lemmalist_ = "".join(lemmalists_[-1:]).split("_")
#                     else :
#                         preverb_ = ""
#                         lemmalist_ = "".join(lemmalists_[0]).split("_")
#                     if(len(lemmalist_) > 1):
#                         auxi_ =auxi_ + " sence of lemma = " + "".join(lemmalist_[1:(len(lemmalist_))])
#                         lemma_ = "".join(lemmalist_[0])
#                     else : 
#                         lemma_ = "".join(lemmalist_[0])             
#                     morphs_ = str(morphslist_[0])               
#                     for morph_ in morphs_.split(" | ") : 
#                         t.loc[id_] = [id_,i,color_,position_,c+1,word_,lemma_,preverb_,morph_,colspan_,len(word_),auxi_]
#                         id_ += 1    


#                 position_ += int(colspan_)
#     #                 break
#             else : 
#                 position_ += 1

# #         break
        
#         i = i+1

        
# # print(color_)
# # print(colspan_)
# # print(position_)
# # print(word_)
# # print(onclickdata_)
# # print(ldata)
# # print(morphslist_)
# # print(lemmalists_)
# # print(auxi_)
# # print(lemma_)
# # print(morphs_)
# # print(preverb_)
# # print(linechar) 
# print(line)
# # c= 0
# # for ch in linechar[0:position_] :
# #     if(re.match('\xa0',ch)) : 
# #         c+=1
# # print(c)
# t